# Chapter 5. Data Class Builders
---

## ToC

[More About @dataclass](#more-about-dataclass)

1. [Field Options](#field-options)  
2. [Post-init Processing](#post-init-processing)
        
---

## More About @dataclass

he decorator accepts several keyword arguments. This is its signature:

```python
@dataclass(*, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False)
```


![Figure 80](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/80.PNG)

`@dataclass` is a convenience tool that automatically generates boilerplate code for classes that primarily store data.
When you use a dataclass, Python auto-generates methods like:

`__init__` — constructor

`__repr__` — developer-friendly string representation

`__eq__` — equality comparison

`__hash__` — hash support (if enabled)

```python
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int
```
Is equivalent to:
```python
class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x and self.y == other.y
```        


**Default Values**

`frozen=True`  
Protects against accidental changes to the class instances.

`order=True`  
Allows sorting of instances of the data class.

If the `eq` and `frozen` arguments are both True, `@dataclass` produces a suitable
`__hash__` method, so the instances will be hashable

In [1]:
from dataclasses import dataclass

@dataclass(frozen=True, eq=False)
class Point:
    x: int
    y: int

p1 = Point(1, 2)
p2 = Point(1, 2)

print(p1 == p2)  # False (uses object identity)
print(hash(p1), hash(p2))  # Same value due to same fields


False
115819066664 115817491226


### Field Options

The instance fields you declare will become parameters in the generated `__init__`. Python does not allow parameters without defaults after parameters with defaults, therefore after you declare a field with a default value, all remaining fields must also have default values.

**Example: Valid Case**

In [1]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str            # No default
    age: int = 30        # Default value provided


This will generate an `__init__` like:

```python
def __init__(self, name: str, age: int = 30):
    ...
```    

**Example: Invalid Case**

In [1]:
from dataclasses import dataclass

@dataclass
class Person:
    age: int = 30        # Default value provided
    name: str            # No default

TypeError: non-default argument 'name' follows default argument 'age'

This will generate an `__init__` like:

```python
def __init__(self, age: int = 30, name: str):
    ...
```

This violates Python’s function signature rule: **non-default arguments must come before default arguments.**

**Mutable default values are a common source of bugs for beginning Python developers.**  
Class attributes are often used as default attribute values for
instances, including in data classes. And `@dataclass` uses the default values in the type hints to generate parameters with defaults for `__init__`.

In [2]:
@dataclass
class ClubMember:
    name: str
    guests: list = []

ValueError: mutable default <class 'list'> for field guests is not allowed: use default_factory

The `ValueError` message explains the problem and suggests a solution: use `default_factory`: lets you provide a function, class, or any other callable, which will be invoked with zero arguments to build a default value each time an instance of the data class is created.

In [6]:
from dataclasses import dataclass, field
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)

More precise:

In [7]:
from dataclasses import dataclass, field
@dataclass
class ClubMember:
    name: str
    guests: list[str] = field(default_factory=list)

In [11]:
alice = ClubMember("Alice", ["Alice's guest"])
bob = ClubMember("Bob", ["Bob's gues"])

alice.guests.append("Charlie")
bob.guests.append("David")

print(alice)
print(bob)

ClubMember(name='Alice', guests=["Alice's guest", 'Charlie'])
ClubMember(name='Bob', guests=["Bob's gues", 'David'])


![Figure 81](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/81.PNG)

![Figure 82](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/82.PNG)

The `default` option exists because the `field` call takes the place of the default value
in the field annotation. If you want to create an `athlete` field with a default value of
`False`, and also omit that field from the `__repr__` method, you’d write this:

In [14]:
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)
    athlete: bool = field(default=False, repr=False)

In [15]:
m = ClubMember("Bob")
print(m)

ClubMember(name='Bob', guests=[])


Without `repr=False`, you'd get:

```python
ClubMember(name='Bob', guests=[], athlete=False)
```

### Post-init Processing